In [ ]:
%pip install pykrx pymysql mysql-connector-python sqlalchemy

In [ ]:
%pip install --upgrade nbstripout

In [ ]:
from pykrx import stock
stock.get_previous_business_days(year=2024, month=10)

In [100]:
import os
from dotenv import load_dotenv
from pykrx import stock
from pykrx import bond

load_dotenv()

KIS_APP_KEY = os.getenv("KIS_APP_KEY")
KIS_APP_SECRET = os.getenv("KIS_APP_SECRET")
KIS_ACC_NO = os.getenv("KIS_ACC_NO")
DB_HOST = os.getenv("DB_HOST")
DB_USERNAME = os.getenv("DB_USERNAME")
DB_PASSWORD = os.getenv("DB_PASSWORD")
DB_SCHEME = os.getenv("DB_SCHEME")
DB_PORT = os.getenv("DB_PORT")


In [101]:
import pandas as pd
import requests as rq
from sqlalchemy import create_engine
from io import StringIO


def get_krx_otp(otp_params, headers):
  krx_gen_otp_url = 'http://data.krx.co.kr/comm/fileDn/GenerateOTP/generate.cmd'
  krx_otp = rq.post(krx_gen_otp_url, otp_params, headers = headers).text
  return krx_otp

def download_krx_index(at_date, market = None):
  if not market:
    kospi_df = download_krx_index(at_date, 'KOSPI')
    kosdaq_df = download_krx_index(at_date, 'KOSDAQ')
    return pd.concat([kospi_df, kosdaq_df])

  headers = {             
    "User-Agent": "Mozilla/5.0",
    "Referer": "http://data.krx.co.kr/"
  }
  otp_params = {
      'locale': 'ko_KR',
      'trdDd': at_date,
      'money': '1',
      'csvxls_isNo': 'false',
      'name': 'fileDown',
      'url': 'dbms/MDC/STAT/standard/MDCSTAT03501'
  }
  if market == 'KOSPI':
    otp_params |= { 'mktId': 'STK' }
  elif market == 'KOSDAQ':
    otp_params |= { 'mktId': 'KSQ', 'segTpCd': 'ALL' }

  otp = get_krx_otp(otp_params, headers)

  download_url = 'http://data.krx.co.kr/comm/fileDn/download_csv/download.cmd'
  download_params = {
      'code': otp
  }
  res = rq.post(download_url, download_params, headers = headers)
  res.encoding = 'euc-kr'
  return pd.read_csv(StringIO(res.text))



def download_krx_sector(at_date, market = None):
  if not market:
    kospi_df = download_krx_sector(at_date, 'KOSPI')
    kosdaq_df = download_krx_sector(at_date, 'KOSDAQ')
    return pd.concat([kospi_df, kosdaq_df])

  headers = {             
    "User-Agent": "Mozilla/5.0",
    "Referer": "http://data.krx.co.kr/"
  }
  otp_params = {
      'locale': 'ko_KR',
      'trdDd': at_date,
      'money': '1',
      'csvxls_isNo': 'false',
      'name': 'fileDown',
      'url': 'dbms/MDC/STAT/standard/MDCSTAT01901'
  }
  if market == 'KOSPI':
    otp_params |= { 'mktId': 'STK' }
  elif market == 'KOSDAQ':
    otp_params |= { 'mktId': 'KSQ', 'segTpCd': 'ALL' }

  otp = get_krx_otp(otp_params, headers)

  download_url = 'http://data.krx.co.kr/comm/fileDn/download_csv/download.cmd'
  download_params = {
      'code': otp
  }
  res = rq.post(download_url, download_params, headers = headers)
  res.encoding = 'euc-kr'
  return pd.read_csv(StringIO(res.text))


def download_stock_category(at_date, market = None):
  if not market:
    kospi_df = download_stock_category(at_date, 'KOSPI')
    kosdaq_df = download_stock_category(at_date, 'KOSDAQ')
    return pd.concat([kospi_df, kosdaq_df])

  headers = {             
    "User-Agent": "Mozilla/5.0",
    "Referer": "http://data.krx.co.kr/"
  }
  otp_params = {
      'locale': 'ko_KR',
      'trdDd': at_date,
      'money': '1',
      'csvxls_isNo': 'false',
      'name': 'fileDown',
      'url': 'dbms/MDC/STAT/standard/MDCSTAT03901'
  }
  if market == 'KOSPI':
    otp_params |= { 'mktId': 'STK' }
  elif market == 'KOSDAQ':
    otp_params |= { 'mktId': 'KSQ', 'segTpCd': 'ALL' }

  otp = get_krx_otp(otp_params, headers)

  download_url = 'http://data.krx.co.kr/comm/fileDn/download_csv/download.cmd'
  download_params = {
      'code': otp
  }
  res = rq.post(download_url, download_params, headers = headers)
  res.encoding = 'euc-kr'
  return pd.read_csv(StringIO(res.text))

def fetchSubCategory():
  engine = create_engine(f'mysql+pymysql://{DB_USERNAME}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_SCHEME}')

  query = "SELECT SUB_CATEGORY_ID, SUB_CATEGORY_NAME FROM SUB_CATEGORY"
  news_data = pd.read_sql(query, engine)

  return news_data

def fetchMainCategory():
  engine = create_engine(f'mysql+pymysql://{DB_USERNAME}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_SCHEME}')

  query = "SELECT MAIN_CATEGORY_ID, MAIN_CATEGORY_NAME FROM MAIN_CATEGORY"
  news_data = pd.read_sql(query, engine)

  return news_data


In [102]:
import datetime

# 현재 날짜를 YYYYMMDD 형식으로 변환
now = datetime.datetime.now()
# current_date_str = now.strftime('%Y%m%d')
current_date_str='20241010'

In [103]:
df_info = download_krx_sector(current_date_str)

In [104]:

df_index = download_krx_index(current_date_str)

In [105]:
sub_category_data = fetchSubCategory()

main_category_data = fetchMainCategory()

In [106]:
stock_category_data = download_stock_category(current_date_str)

In [107]:
stock_category_data = stock_category_data[['종목코드', '업종명']]

In [108]:
stock_category_data.rename(columns={'종목코드':'SHORT_CODE','업종명':'SUB_CATEGORY_NAME'}, inplace=True)

In [ ]:
stock_category_data

In [ ]:
sub_category_data

In [111]:
merged_stock_category = pd.merge(stock_category_data, sub_category_data, on='SUB_CATEGORY_NAME', how='inner')

In [ ]:
merged_stock_category

In [ ]:
df_info

In [ ]:
df_index

In [115]:
df_info_mapping = {
    '단축코드': 'SHORT_CODE',
}

df_index_mapping = {
    '종목코드': 'SHORT_CODE',
}

# 컬럼명 매핑 적용
df_info.rename(columns=df_info_mapping, inplace=True)
df_index.rename(columns=df_index_mapping, inplace=True)

In [116]:
df_info_index = pd.merge(df_info, df_index, on='SHORT_CODE', how='outer')
df_info_index_stock_category = pd.merge(df_info_index, merged_stock_category, on="SHORT_CODE", how='inner')

In [ ]:
df_info_index_stock_category

In [118]:
merged_df_column_mapping = {
    '단축코드': 'SHORT_CODE',
    # '서브카테고리' : 'SUB_CATEGORY_ID',
    '한글 종목약명': 'STOCK_NAME',
    '표준코드': 'STANDARD_CODE',
    '시장구분': 'STOCK_EXCHANGE_MARKET',
    '상장주식수': 'MARKET_CAPITALIZATION',
    '시가': 'OPEN_PRICE',
    '종가': 'CLOSED_PRICE',
    '고가': 'HIGH_PRICE',
    '저가': 'LOW_PRICE',
    '거래정지': 'STOCK_TRADE_STATUS',
}


df_info_index_stock_category.rename(columns=merged_df_column_mapping, inplace=True)

In [119]:

real_index = [
 'SHORT_CODE',
 'STOCK_NAME',
 'SUB_CATEGORY_ID',
 'STANDARD_CODE',
 'STOCK_EXCHANGE_MARKET',
 'MARKET_CAPITALIZATION',
 'EPS',
 'PER',
 'BPS',
 'PBR',
]

df_info_index_stock_category = df_info_index_stock_category[real_index]

In [ ]:
df_info_index_stock_category

In [ ]:
# 해당 데이터가 확보 안된 경우 사용
for column in [
 'STOCK_TRADE_STATUS']:
    if column not in df_info_index_stock_category.columns:
        df_info_index_stock_category[column] = 'A'


In [ ]:
df_info_index_stock_category['STOCK_EXCHANGE_MARKET'] = df_info_index_stock_category['STOCK_EXCHANGE_MARKET'].map({'KOSPI': "0", 'KOSDAQ': "1", "KOSDAQ GLOBAL" : "1"})

In [ ]:
df_info_index_stock_category

In [124]:
df_info_index_stock_category = df_info_index_stock_category[df_info_index_stock_category['STANDARD_CODE'].notna()]

In [ ]:
from sqlalchemy import create_engine
from sqlalchemy import text
# MySQL 데이터베이스 연결 설정
# db_username = DB_USERNAME
# db_password = DB_PASSWORD
# db_host = DB_HOST
# db_name = DB_SCHEME
# db_port = DB_PORT

# SQLAlchemy 엔진 생성
engine = create_engine(f'mysql+pymysql://{DB_USERNAME}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_SCHEME}')

df_info_index_stock_category.to_sql(name='TEMP_STOCK', con=engine, if_exists='append', index=False)

# 업데이트 쿼리 작성
update_query = """
INSERT INTO STOCK (SHORT_CODE, SUB_CATEGORY_ID, STOCK_NAME, STANDARD_CODE, STOCK_EXCHANGE_MARKET, 
                   EPS, PER, BPS, PBR, MARKET_CAPITALIZATION, STOCK_TRADE_STATUS, UPDATED_AT)
SELECT SHORT_CODE, SUB_CATEGORY_ID, STOCK_NAME, STANDARD_CODE, STOCK_EXCHANGE_MARKET, 
       EPS, PER, BPS, PBR, MARKET_CAPITALIZATION, STOCK_TRADE_STATUS, NOW()
FROM TEMP_STOCK
ON DUPLICATE KEY UPDATE 
    SUB_CATEGORY_ID = VALUES(SUB_CATEGORY_ID),
    STOCK_NAME = VALUES(STOCK_NAME),
    STANDARD_CODE = VALUES(STANDARD_CODE),
    STOCK_EXCHANGE_MARKET = VALUES(STOCK_EXCHANGE_MARKET),
    EPS = VALUES(EPS),
    PER = VALUES(PER),
    BPS = VALUES(BPS),
    PBR = VALUES(PBR),
    MARKET_CAPITALIZATION = VALUES(MARKET_CAPITALIZATION),
    STOCK_TRADE_STATUS = VALUES(STOCK_TRADE_STATUS),
    UPDATED_AT = NOW();
"""

# SQL 쿼리 실행
with engine.connect() as connection:
    connection.execute(text(update_query))

# 임시 테이블 삭제
# with engine.connect() as connection:
#     connection.execute(text("DROP TABLE TEMP_STOCK"))
